Service Mesh - Istio
--------------------

![](https://istio.io/latest/docs/ops/deployment/architecture/arch.svg)

Source: istio.io

- - -

Cloud platforms offer numerous advantages to the companies that use them.

To do this, developers have to use the microservices architecture pattern, which means that different versions of microservices quickly have to be operated in parallel.

With the rich capabilities of [Istio](http://istio.io), one can successfully and efficiently run a distributed microservices architecture and secure, connect, and monitor microservices in a unified way.

It is a fully open source service mesh built on top of existing distributed applications.

***
### What is a service mesh?

The term service mesh describes the network of microservices and the interactions between them.

The larger and more complex a service mesh becomes, the more difficult it is to understand and manage.

Requirements include discovery, load balancing, troubleshooting, metrics, and monitoring.

A service mesh often has more complex operational requirements, such as A/B testing (evaluating two variants), canary rollouts, restrictions, access control, and end-to-end authentication.

More information: [Istio Homepage](https://istio.io/docs/concepts/what-is-istio/)

***
### Installation

Istio is already installed in the [duk-demo](https://github.com/mc-b/duk-demo) environment. Otherwise see project [duk-demo](https://github.com/mc-b/duk-demo).

***
### Book info example

The sample application displays information about a book, much like a single catalog listing from an online bookstore. The page shows a description of the book, book details (ISBN, page number, etc.), and some book reviews.

See [Bookinfo Application](https://istio.io/docs/examples/bookinfo/) and the [Bookinfo Application](https://istio.io/docs/examples/bookinfo/) for details.

The sample application is part of the Istio sources. Therefore we need to download this first, then we create our own namespace with istio enabled and inside it the Bookinfo sample application.


In [ ]:
! kubectl create namespace bookinfo
! kubectl label namespace bookinfo istio-injection=enabled
! kubectl apply -f https://raw.githubusercontent.com/istio/istio/release-1.13/samples/bookinfo/platform/kube/bookinfo.yaml -n bookinfo
! kubectl apply -f https://raw.githubusercontent.com/istio/istio/release-1.13/samples/bookinfo/networking/bookinfo-gateway.yaml -n bookinfo
! kubectl apply -f https://raw.githubusercontent.com/istio/istio/release-1.13/samples/bookinfo/networking/destination-rule-all.yaml -n bookinfo

In addition to pods, services, etc., other resources were also created such as:
* [Gateway](https://istio.io/docs/concepts/traffic-management/#gateways) - for access from outside the cluster
* [Virtual Service](https://istio.io/docs/concepts/traffic-management/#virtual-services) - mit den URL Paths
* [Routing Rule](https://istio.io/docs/concepts/traffic-management/#routing-rules) - with the rules what is forwarded where.


In [ ]:
! kubectl get all,gw,vs,dr -n bookinfo

In [ ]:
! kubectl -n bookinfo get destinationrules -o yaml

But let's look at the application first.

![](https://istio.io/latest/docs/examples/bookinfo/noistio.svg)

Source: istio.io

- - -

The Bookinfo application is divided into four separate microservices:

* productpage: The productpage microservice calls the details and reviews microservices to populate the page.
* details: The details microservice contains book information.
* reviews: The reviews microservice contains book reviews. It also calls the ratings microservice.
* ratings: The ratings microservice contains book rank information attached to a book review.

There are 3 versions of the reviews microservice:
* Version v1 does not call the ratings microservice.
* Version v2 calls the ratings microservice and displays each rating as 1 to 5 black stars.
* Version v3 calls the ratings microservice and displays each rating as 1 to 5 red stars.

This application are implemented in several programming language.

The microservices have no dependencies and are not aware of Istio. They have also not been specifically modified to work with Istio.

The user interface can be selected as follows:

In [ ]:
%%bash
[ ! -f ~/work/server-ip ] && { echo "replace-with-server-ip" >~/work/server-ip; }
PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="http2")].nodePort}')
echo http://$(cat ~/work/server-ip):$PORT/productpage

If we select the page several times, one of the three versions of the `reviews` microservice is called up alternately.

To target:

* access only one version, version 1 or 3
* Call version 2 and 3 analogous to a canary rollout 50/50 percent

prepared YAML files are available. Run each one individually and test the effect in the UI. It can take 1-2 minutes for the rules to become active.

In [ ]:
! kubectl apply -f https://raw.githubusercontent.com/istio/istio/release-1.13/samples/bookinfo/networking/virtual-service-all-v1.yaml -n bookinfo

In [ ]:
! kubectl apply -f https://raw.githubusercontent.com/istio/istio/release-1.13/samples/bookinfo/networking/virtual-service-reviews-v3.yaml -n bookinfo

In [ ]:
! kubectl apply -f https://raw.githubusercontent.com/istio/istio/release-1.13/samples/bookinfo/networking/virtual-service-reviews-v2-v3.yaml -n bookinfo

Clean up

In [ ]:
! kubectl delete namespace bookinfo